<a href="https://colab.research.google.com/github/chuckyLeeVIII/AUTO-RBF-BTC-ETH/blob/main/Standard_CPFP_simple_cpfp_tool_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyQt6

import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton,
    QTextEdit, QLabel, QLineEdit, QGroupBox, QFormLayout, QMessageBox
)
from PyQt6.QtCore import QThread, pyqtSignal

# ==========================================
# CONSTANTS (Standard Bitcoin Protocol)
# ==========================================
API_URL = "https://blockstream.info/api"
ESTIMATED_CHILD_VBYTES = 141  # Standard P2WPKH Input + Output + Overhead
MIN_RELAY_FEE = 200  # Sats

# ==========================================
# WORKER THREAD (Network Calls)
# ==========================================
class NetworkWorker(QThread):
    data_signal = pyqtSignal(bool, dict)

    def __init__(self, txid):
        super().__init__()
        self.txid = txid

    def run__(self):
        try:
            # Fetch Parent TX Data
            r = requests.get(f"{API_URL}/tx/{self.txid}", timeout=10)
            if r.status_code != 200:
                self.data_signal.emit(False, {"error": "Transaction not found."})
                return

            data = r.json()
            # Fetch Output Data to verify amount
            self.data_signal.emit(True, data)
        except Exception as e:
            self.data_signal.emit(False, {"error": str(e)})

# ==========================================
# MAIN APPLICATION
# ==========================================
class SimpleCPFP(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Simple Bitcoin CPFP Tool")
        self.resize(500, 650)
        self.setStyleSheet("""
            QMainWindow { background-color: #f0f0f0; }
            QGroupBox { font-weight: bold; border: 1px solid #ccc; margin-top: 10px; background: white; }
            QLineEdit, QTextEdit { padding: 8px; border: 1px solid #bbb; }
            QPushButton { background-color: #2c3e50; color: white; padding: 10px; font-weight: bold; }
            QPushButton:hover { background-color: #34495e; }
        """)

        self.current_parent_data = None
        self.init_ui()

    def init_ui(self):
        container = QWidget()
        layout = QVBoxLayout()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # HEADER
        layout.addWidget(QLabel("<h2>Bitcoin Child-Pays-For-Parent</h2>"))

        # SECTION 1: STUCK TRANSACTION
        g1 = QGroupBox("1. Stuck Transaction Details")
        f1 = QFormLayout()

        self.inp_txid = QLineEdit()
        self.inp_txid.setPlaceholderText("Paste Parent TXID here...")

        self.inp_vout = QLineEdit("0")
        self.inp_vout.setPlaceholderText("Output Index (usually 0 or 1)")

        self.btn_load = QPushButton("Load Transaction Data")
        self.btn_load.clicked.connect(self.load_tx_data)

        f1.addRow("Parent TXID:", self.inp_txid)
        f1.addRow("Output Index (Vout):", self.inp_vout)
        f1.addRow("", self.btn_load)
        g1.setLayout(f1)
        layout.addWidget(g1)

        # SECTION 2: DESTINATION
        g2 = QGroupBox("2. Where are funds going?")
        f2 = QFormLayout()

        self.inp_address = QLineEdit()
        self.inp_address.setPlaceholderText("New Deposit Address (bc1... or 1...)")

        self.inp_fee_rate = QLineEdit("50")
        self.inp_fee_rate.setPlaceholderText("Target Fee Rate (sat/vB)")

        f2.addRow("New Address:", self.inp_address)
        f2.addRow("Target Fee Rate:", self.inp_fee_rate)
        g2.setLayout(f2)
        layout.addWidget(g2)

        # ACTION
        self.btn_calc = QPushButton("GENERATE CPFP VALUES")
        self.btn_calc.clicked.connect(self.calculate_cpfp)
        self.btn_calc.setEnabled(False)
        self.btn_calc.setStyleSheet("background-color: #e67e22; color: white;")
        layout.addWidget(self.btn_calc)

        # RESULTS
        self.txt_result = QTextEdit()
        self.txt_result.setPlaceholderText("Results will appear here...")
        layout.addWidget(self.txt_result)

    def load_tx_data(self):
        txid = self.inp_txid.text().strip()
        if not txid: return

        self.txt_result.setText("Fetching data from Bitcoin network...")
        self.worker = NetworkWorker(txid)
        self.worker.data_signal.connect(self.on_data_loaded)
        self.worker.start()

    def on_data_loaded(self, success, data):
        if not success:
            QMessageBox.critical(self, "Error", data['error'])
            self.txt_result.setText("")
            return

        self.current_parent_data = data
        fee = data['fee']
        weight = data['weight']

        info = (f"✔ Parent Loaded\n"
                f"   Fee Paid: {fee} sats\n"
                f"   Weight: {weight} wu ({weight/4:.1f} vB)\n"
                f"   Confirmed: {data['status']['confirmed']}")

        self.txt_result.setText(info)
        self.btn_calc.setEnabled(True)

    def calculate_cpfp(self):
        if not self.current_parent_data: return

        try:
            # 1. Gather Inputs
            parent_fee_paid = self.current_parent_data['fee']
            parent_weight = self.current_parent_data['weight'] # Weight Units

            target_rate = float(self.inp_fee_rate.text())
            target_vout = int(self.inp_vout.text())
            new_address = self.inp_address.text().strip()

            # Check if Vout exists
            if target_vout >= len(self.current_parent_data['vout']):
                self.txt_result.setText("Error: Output Index (Vout) does not exist in Parent TX.")
                return

            input_amount_sats = self.current_parent_data['vout'][target_vout]['value']

            # 2. THE MATH
            # Package Weight = Parent Weight + Child Weight (Est)
            # We calculate in vBytes for the rate, but use Weight for precision
            child_weight_est = ESTIMATED_CHILD_VBYTES * 4
            total_package_weight = parent_weight + child_weight_est
            total_package_vbytes = total_package_weight / 4

            # Total fee required for the whole package
            total_fee_needed = total_package_vbytes * target_rate

            # Deficit = Total Needed - Parent Paid
            child_fee_sats = int(total_fee_needed - parent_fee_paid)

            # Safety Floor
            child_fee_sats = max(child_fee_sats, MIN_RELAY_FEE)

            # Amount to send to New Deposit Address
            amount_to_send = input_amount_sats - child_fee_sats

            if amount_to_send <= 0:
                self.txt_result.setText("Error: Fee required is higher than the available balance.")
                return

            # 3. OUTPUT
            res = (
                "=== CPFP INSTRUCTIONS ===\n\n"
                "To unstick the transaction, create a NEW transaction with these exact values:\n\n"
                f"INPUT:\n"
                f"  TxID: {self.inp_txid.text()}\n"
                f"  Vout: {target_vout}\n\n"
                f"OUTPUT (New Deposit Address):\n"
                f"  Address: {new_address}\n"
                f"  Amount:  {amount_to_send} sats ({amount_to_send/100000000:.8f} BTC)\n\n"
                f"FEE:\n"
                f"  Fee:     {child_fee_sats} sats ({child_fee_sats/100000000:.8f} BTC)\n\n"
                f"--------------------------\n"
                f"Effective Package Rate: {target_rate} sat/vB"
            )

            self.txt_result.setText(res)

        except ValueError:
            self.txt_result.setText("Error: Please check your numbers.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = SimpleCPFP()
    window.show()
    sys.exit(app.exec())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.3/304.3 kB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install PyQt6

In [ ]:
!pip install PyQt6

import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton,
    QTextEdit, QLabel, QLineEdit, QGroupBox, QFormLayout, QMessageBox
)
from PyQt6.QtCore import QThread, pyqtSignal

# ==========================================
# CONSTANTS (Standard Bitcoin Protocol)
# ==========================================
API_URL = "https://blockstream.info/api"
ESTIMATED_CHILD_VBYTES = 141  # Standard P2WPKH Input + Output + Overhead
MIN_RELAY_FEE = 200  # Sats

# ==========================================
# WORKER THREAD (Network Calls)
# ==========================================
class NetworkWorker(QThread):
    data_signal = pyqtSignal(bool, dict)

    def __init__(self, txid):
        super().__init__()
        self.txid = txid

    def run__(self):
        try:
            # Fetch Parent TX Data
            r = requests.get(f"{API_URL}/tx/{self.txid}", timeout=10)
            if r.status_code != 200:
                self.data_signal.emit(False, {"error": "Transaction not found."})
                return

            data = r.json()
            # Fetch Output Data to verify amount
            self.data_signal.emit(True, data)
        except Exception as e:
            self.data_signal.emit(False, {"error": str(e)})

# ==========================================
# MAIN APPLICATION
# ==========================================
class SimpleCPFP(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Simple Bitcoin CPFP Tool")
        self.resize(500, 650)
        self.setStyleSheet("""
            QMainWindow { background-color: #f0f0f0; }
            QGroupBox { font-weight: bold; border: 1px solid #ccc; margin-top: 10px; background: white; }
            QLineEdit, QTextEdit { padding: 8px; border: 1px solid #bbb; }
            QPushButton { background-color: #2c3e50; color: white; padding: 10px; font-weight: bold; }
            QPushButton:hover { background-color: #34495e; }
        """)

        self.current_parent_data = None
        self.init_ui()

    def init_ui(self):
        container = QWidget()
        layout = QVBoxLayout()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # HEADER
        layout.addWidget(QLabel("<h2>Bitcoin Child-Pays-For-Parent</h2>"))

        # SECTION 1: STUCK TRANSACTION
        g1 = QGroupBox("1. Stuck Transaction Details")
        f1 = QFormLayout()

        self.inp_txid = QLineEdit()
        self.inp_txid.setPlaceholderText("Paste Parent TXID here...")

        self.inp_vout = QLineEdit("0")
        self.inp_vout.setPlaceholderText("Output Index (usually 0 or 1)")

        self.btn_load = QPushButton("Load Transaction Data")
        self.btn_load.clicked.connect(self.load_tx_data)

        f1.addRow("Parent TXID:", self.inp_txid)
        f1.addRow("Output Index (Vout):", self.inp_vout)
        f1.addRow("", self.btn_load)
        g1.setLayout(f1)
        layout.addWidget(g1)

        # SECTION 2: DESTINATION
        g2 = QGroupBox("2. Where are funds going?")
        f2 = QFormLayout()

        self.inp_address = QLineEdit()
        self.inp_address.setPlaceholderText("New Deposit Address (bc1... or 1...)")

        self.inp_fee_rate = QLineEdit("50")
        self.inp_fee_rate.setPlaceholderText("Target Fee Rate (sat/vB)")

        f2.addRow("New Address:", self.inp_address)
        f2.addRow("Target Fee Rate:", self.inp_fee_rate)
        g2.setLayout(f2)
        layout.addWidget(g2)

        # ACTION
        self.btn_calc = QPushButton("GENERATE CPFP VALUES")
        self.btn_calc.clicked.connect(self.calculate_cpfp)
        self.btn_calc.setEnabled(False)
        self.btn_calc.setStyleSheet("background-color: #e67e22; color: white;")
        layout.addWidget(self.btn_calc)

        # RESULTS
        self.txt_result = QTextEdit()
        self.txt_result.setPlaceholderText("Results will appear here...")
        layout.addWidget(self.txt_result)

    def load_tx_data(self):
        txid = self.inp_txid.text().strip()
        if not txid: return

        self.txt_result.setText("Fetching data from Bitcoin network...")
        self.worker = NetworkWorker(txid)
        self.worker.data_signal.connect(self.on_data_loaded)
        self.worker.start()

    def on_data_loaded(self, success, data):
        if not success:
            QMessageBox.critical(self, "Error", data['error'])
            self.txt_result.setText("")
            return

        self.current_parent_data = data
        fee = data['fee']
        weight = data['weight']

        info = (f"✔ Parent Loaded\n"
                f"   Fee Paid: {fee} sats\n"
                f"   Weight: {weight} wu ({weight/4:.1f} vB)\n"
                f"   Confirmed: {data['status']['confirmed']}")

        self.txt_result.setText(info)
        self.btn_calc.setEnabled(True)

    def calculate_cpfp(self):
        if not self.current_parent_data: return

        try:
            # 1. Gather Inputs
            parent_fee_paid = self.current_parent_data['fee']
            parent_weight = self.current_parent_data['weight'] # Weight Units

            target_rate = float(self.inp_fee_rate.text())
            target_vout = int(self.inp_vout.text())
            new_address = self.inp_address.text().strip()

            # Check if Vout exists
            if target_vout >= len(self.current_parent_data['vout']):
                self.txt_result.setText("Error: Output Index (Vout) does not exist in Parent TX.")
                return

            input_amount_sats = self.current_parent_data['vout'][target_vout]['value']

            # 2. THE MATH
            # Package Weight = Parent Weight + Child Weight (Est)
            # We calculate in vBytes for the rate, but use Weight for precision
            child_weight_est = ESTIMATED_CHILD_VBYTES * 4
            total_package_weight = parent_weight + child_weight_est
            total_package_vbytes = total_package_weight / 4

            # Total fee required for the whole package
            total_fee_needed = total_package_vbytes * target_rate

            # Deficit = Total Needed - Parent Paid
            child_fee_sats = int(total_fee_needed - parent_fee_paid)

            # Safety Floor
            child_fee_sats = max(child_fee_sats, MIN_RELAY_FEE)

            # Amount to send to New Deposit Address
            amount_to_send = input_amount_sats - child_fee_sats

            if amount_to_send <= 0:
                self.txt_result.setText("Error: Fee required is higher than the available balance.")
                return

            # 3. OUTPUT
            res = (
                "=== CPFP INSTRUCTIONS ===\n\n"
                "To unstick the transaction, create a NEW transaction with these exact values:\n\n"
                f"INPUT:\n"
                f"  TxID: {self.inp_txid.text()}\n"
                f"  Vout: {target_vout}\n\n"
                f"OUTPUT (New Deposit Address):\n"
                f"  Address: {new_address}\n"
                f"  Amount:  {amount_to_send} sats ({amount_to_send/100000000:.8f} BTC)\n\n"
                f"FEE:\n"
                f"  Fee:     {child_fee_sats} sats ({child_fee_sats/100000000:.8f} BTC)\n\n"
                f"--------------------------\n"
                f"Effective Package Rate: {target_rate} sat/vB"
            )

            self.txt_result.setText(res)

        except ValueError:
            self.txt_result.setText("Error: Please check your numbers.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = SimpleCPFP()
    window.show()
    sys.exit(app.exec())